In [1]:
%matplotlib inline
import numpy as np
import tflearn

# Download the Titanic dataset
from tflearn.datasets import titanic
titanic.download_dataset('titanic_dataset.csv')

# Load CSV file, indicate that the first column represents labels
from tflearn.data_utils import load_csv
data, labels = load_csv('titanic_dataset.csv', target_column=0,
                        categorical_labels=True, n_classes=2)

In [2]:
data

[['1',
  'Allen, Miss. Elisabeth Walton',
  'female',
  '29',
  '0',
  '0',
  '24160',
  '211.3375'],
 ['1',
  'Allison, Master. Hudson Trevor',
  'male',
  '0.9167',
  '1',
  '2',
  '113781',
  '151.5500'],
 ['1',
  'Allison, Miss. Helen Loraine',
  'female',
  '2',
  '1',
  '2',
  '113781',
  '151.5500'],
 ['1',
  'Allison, Mr. Hudson Joshua Creighton',
  'male',
  '30',
  '1',
  '2',
  '113781',
  '151.5500'],
 ['1',
  'Allison, Mrs. Hudson J C (Bessie Waldo Daniels)',
  'female',
  '25',
  '1',
  '2',
  '113781',
  '151.5500'],
 ['1', 'Anderson, Mr. Harry', 'male', '48', '0', '0', '19952', '26.5500'],
 ['1',
  'Andrews, Miss. Kornelia Theodosia',
  'female',
  '63',
  '1',
  '0',
  '13502',
  '77.9583'],
 ['1', 'Andrews, Mr. Thomas Jr', 'male', '39', '0', '0', '112050', '0.0000'],
 ['1',
  'Appleton, Mrs. Edward Dale (Charlotte Lamson)',
  'female',
  '53',
  '2',
  '0',
  '11769',
  '51.4792'],
 ['1',
  'Artagaveytia, Mr. Ramon',
  'male',
  '71',
  '0',
  '0',
  'PC 17609',
  '49

In [6]:
# Preprocessing function
def preprocess(data, columns_to_ignore):
    # Sort by descending id and delete columns
    for columns_to_ignore in sorted(columns_to_ignore, reverse=True):
        [r.pop(columns_to_ignore) for r in data]
    for i in range(len(data)):
      # Converting 'sex' field to float (id is 1 after removing labels column)
      data[i][1] = 1. if data[i][1] == 'female' else 0.
    return np.array(data, dtype=np.float32)

# Ignore 'name' and 'ticket' columns (id 1 & 6 of data array)
to_ignore=[1, 6]

# Preprocess data
data = preprocess(data, to_ignore)

AttributeError: 'numpy.ndarray' object has no attribute 'pop'

In [4]:
data

array([[   1.        ,    1.        ,   29.        ,    0.        ,
           0.        ,  211.3374939 ],
       [   1.        ,    0.        ,    0.91670001,    1.        ,
           2.        ,  151.55000305],
       [   1.        ,    1.        ,    2.        ,    1.        ,
           2.        ,  151.55000305],
       ..., 
       [   3.        ,    0.        ,   26.5       ,    0.        ,
           0.        ,    7.2249999 ],
       [   3.        ,    0.        ,   27.        ,    0.        ,
           0.        ,    7.2249999 ],
       [   3.        ,    0.        ,   29.        ,    0.        ,
           0.        ,    7.875     ]], dtype=float32)

In [5]:
# Build neural network
net = tflearn.input_data(shape=[None, 6])
net = tflearn.fully_connected(net, 32)
net = tflearn.fully_connected(net, 32)
net = tflearn.fully_connected(net, 2, activation='softmax')
net = tflearn.regression(net)

In [6]:
# Define model
model = tflearn.DNN(net, tensorboard_verbose=3)
# Start training (apply gradient descent algorithm)
model.fit(data, labels, n_epoch=10, batch_size=16, show_metric=True)

Training Step: 819  | total loss: 0.47639 | time: 2.228s
| Adam | epoch: 010 | loss: 0.47639 - acc: 0.7871 -- iter: 1296/1309
Training Step: 820  | total loss: 0.46451 | time: 2.254s
| Adam | epoch: 010 | loss: 0.46451 - acc: 0.8021 -- iter: 1309/1309
--


In [7]:
# Let's create some data for DiCaprio and Winslet
dicaprio = [3, 'Jack Dawson', 'male', 19, 0, 0, 'N/A', 5.0000]
winslet = [1, 'Rose DeWitt Bukater', 'female', 17, 1, 2, 'N/A', 100.0000]
# Preprocess data
dicaprio, winslet = preprocess([dicaprio, winslet], to_ignore)
# Predict surviving chances (class 1 results)
pred = model.predict([dicaprio, winslet])
print("DiCaprio Surviving Rate:", pred[0][1])
print("Winslet Surviving Rate:", pred[1][1])

DiCaprio Surviving Rate: 0.13661682605743408
Winslet Surviving Rate: 0.8336880803108215


In [8]:
pred

[[0.8633831143379211, 0.13661682605743408],
 [0.1663118451833725, 0.8336880803108215]]

/bin/sh: tensorboard: command not found
